# Refactor Public Goods Game


## Overall TODOS

In [ ]:
# Added specified player type proportions (population wide / or within groups if I am staying with group_size == 4 num_membership == 2)
# Completed 

#TODO Implement improved player-types (altruist and freerider) for split endowment case (make them a little more responsive versus totally static and identical) 
# aka varying degrees of altruism : debating this one – for behavioral / lab results reasons 

#DID Add player types and improved decision rule(s) for shared endowment case 
# Started on. The current decison rule works. Want to improve it even more
# M-20 Improved on it even more. Added between-group comparison for SHARED endowment.

#TODO
# I now want to change the type of randomness used. Rather than a uniform distribution, maybe something with a tail, etc. 

#TODO Behavior Issue. Check current output. Currently, very drastic changes in the first round, but then almost no changes occur later on. This should be worked on.

#TODO Additional issue : I do not think my scale factor is not working properly

#DID Initialize contributions for SHARED endowment case with player types
# Completed

#DID Fix Simulate Game Function for Shared Endowment Case
# Completed

#DID Improve the way my csv file looks and make sure it works with the monte carlo simulation portion (I think I am only saving the final iterations data as it stands)
# Completed. It now saves data from all iterations of the Simulation. HOWEVER !!!

## Completed Switch from numpyarray for MOI ==> Pandas DF. to_csv, as well as naming the rows/columns will be of
# great importance for my capacity to use the data in the future 
# Did, but now I cannot open the CSV FILE PROPERLY

#TODO Once I do the above, I can make better graphs that show (1) average group level cooperation, (2) average player level coop, (3) average round level coop, etc.

#TODO design in a way to allow for specific comparissons between the nature of the endowment (SHARED == 0/1) with same parameter sets 

#TODO parameterize to explore the parameter space

#TODO implement power analysis by estimating effect size given the information that I am storing 

#TODO I would be interested in creating some visual component for displaying the change of cooperation of time given different parameter sets -- potentially with
# a scroller feature 

# Import Utils

In [1]:
from src.public_goods_utils import *
import matplotlib.pyplot as plt

In [ ]:
groups_example = formed_groups_matrix(GROUP_SIZE, NUM_GROUPS)
np.argwhere(groups_example == 1) # GOOD FOR VECTORIZING LATER; gives me indices of what groups ppl are in. Will no longer need to use for loop

## & Graphs 
- Working on Improving the graphs, what information I can display, etc.
- Currently working on in 'tests.py' will be removed from here once I have a better version.

In [ ]:
x = [i for i in range(NUM_ROUNDS)]
y = example_MC[1][:-1]
plt.title(f"Endowment: {ENDOWMENT}, Shared: {SHARED}, In How many groups?: {NUM_MEMBERSHIP}, Group Size: {GROUP_SIZE}, Sims: {SIMS}")
plt.xlabel("Round")
plt.ylabel("Expected Round Contribution")
plt.plot(x, y, color ="green")
plt.show()